# quick start

In [1]:
import numpy as np
import pandas as pd

from dask.distributed import Client

from condor_utils import make_htcondor_cluster

In [2]:
cluster = make_htcondor_cluster(local=False, dashboard_address=13346)
cluster

In [147]:
cluster.scale(100)

In [4]:
client = Client(cluster)
client

Client Scheduler: tcp://169.228.130.37:23134 Dashboard: http://169.228.130.37:13346/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Set up environment for root and combine

In [11]:
def f():
    import subprocess

    script="""
source /cvmfs/cms.cern.ch/cmsset_default.sh 
cd /cvmfs/cms.cern.ch/slc7_amd64_gcc700/cms/cmssw/CMSSW_10_5_0/ 
eval `scramv1 runtime -sh` 
cd - 
root --help

cp /hadoop/cms/store/user/hmei/combineStandalone.tar.gz .
tar -xzf combineStandalone.tar.gz
pwd

# https://root-forum.cern.ch/t/error-in-cling-insertintoautoloadingstate/29347
# Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState
export ROOT_INCLUDE_PATH=/srv/temp/

cd HiggsAnalysis/CombinedLimit/
source ./env_standalone.sh 
combine -h

#which combine
"""
    with open("temp.sh","w") as fh:
        fh.write(script)
    _ = subprocess.getoutput("chmod u+x temp.sh")
    cmd = "time ./temp.sh"
    return subprocess.getoutput(cmd)
#print(client.submit(f).result())

In [12]:
# Run a function on all workers outside of task scheduling system
#%time
client.run(f)

{'tcp://169.228.130.139:44981': '/srv/temp\n\nUsage: root [-l] [-b] [-n] [-q] [dir] [[file:]data.root] [file1.C ... fileN.C]\nOptions:\n  -b : run in batch mode without graphics\n  -x : exit on exception\n  -e expression: request execution of the given C++ expression\n  -n : do not execute logon and logoff macros as specified in .rootrc\n  -q : exit after processing command line macro files\n  -l : do not show splash screen\n  -t : enable thread-safety and implicit multi-threading (IMT)\n --notebook : execute ROOT notebook\n dir : if dir is a valid directory cd to it before executing\n\n  -?      : print usage\n  -h      : print usage\n  --help  : print usage\n  -config : print ./configure options\n  -memstat : run with memory usage monitoring\n\n/srv/temp\nUsage: combine datacard [options]\nMain options:\n  -d [ --datacard ] arg                 Datacard file (can also be specified \n                                        directly without the -d or --datacard)\n  -M [ --method ] arg (

In [18]:
def testCombine():
    from subprocess import call
    #call("cp /hadoop/cms/store/user/hmei/testCard.txt .", shell=True)
    #call("combine -M AsymptoticLimits testCard.txt >> result.txt", shell=True)
    call("ls . >> result.txt", shell=True)
    #call("which combine > result.txt", shell=True)
    with open('result.txt', 'r') as file:
        data = file.read().replace('\n', '')
    return data

In [19]:
futures = client.submit(testCombine)

In [20]:
#client.who_has([futures3])
#futures3
futures.result()
#count = futures.result()

'HiggsAnalysiscachepreload.pycombineStandalone.tar.gzglobal.lockls.txtpurge.lockresult.txttemp.shutils.pyworker-r144jda9worker-r144jda9.dirlockworkerenvworkerenv.tar.gz'

In [106]:
def g(replace):
    import subprocess

    script="""
source /cvmfs/cms.cern.ch/cmsset_default.sh 
cd /cvmfs/cms.cern.ch/slc7_amd64_gcc700/cms/cmssw/CMSSW_10_5_0/ 
eval `scramv1 runtime -sh` 
cd - 
#root --help

cp /hadoop/cms/store/user/hmei/combineStandalone.tar.gz .
tar -xzf combineStandalone.tar.gz
#pwd

# https://root-forum.cern.ch/t/error-in-cling-insertintoautoloadingstate/29347
# Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState
export ROOT_INCLUDE_PATH=/srv/temp/

cd HiggsAnalysis/CombinedLimit/
source ./env_standalone.sh 
#combine -h

cp /hadoop/cms/store/user/hmei/testCard.txt .
combine -M AsymptoticLimits testCard.txt
#which combine
"""

    with open("temp.sh","w") as fh:
        fh.write(script)
    _ = subprocess.getoutput("chmod u+x temp.sh")
    cmd = "time ./temp.sh"
    return subprocess.getoutput(cmd)

In [169]:
#futures2 = client.submit(testCombine2)
futures3 = client.map(g,range(1,200))

In [213]:
futures3

[<Future: finished, type: builtins.str, key: g-21947000c2158e2d2fe0385e696b6646>,
 <Future: finished, type: builtins.str, key: g-aac5d2feddcf18bd765aced6b337bca3>,
 <Future: finished, type: builtins.str, key: g-1510f0b40dea3c957bc5925d24cafa32>,
 <Future: finished, type: builtins.str, key: g-d7564254ff59fadb47a046779e27281e>,
 <Future: finished, type: builtins.str, key: g-80a049a3625e06c6d0e26a63165b51a0>,
 <Future: finished, type: builtins.str, key: g-82cd84ea39856f08c7d7ec299a16b293>,
 <Future: finished, type: builtins.str, key: g-597fd9aef173ecdb7b3f5b7759c5e8e6>,
 <Future: finished, type: builtins.str, key: g-0bbad070cd423a7b68b6e1c5485af0f7>,
 <Future: finished, type: builtins.str, key: g-1131e152092961c77f25cf513dada5f2>,
 <Future: finished, type: builtins.str, key: g-800c08484128a7a25a988e65da68adcb>,
 <Future: finished, type: builtins.str, key: g-823f46955cb547a46d6561179cef0049>,
 <Future: finished, type: builtins.str, key: g-f753da42e015b0ab746f6beee50fc658>,
 <Future: finish

In [146]:
#futures2.result()
results = client.gather(futures3)
results
#tmp = futures2.result().split("\n")
#for i in range(len(tmp)):
#    print (tmp[i])

['/srv/temp\n <<< Combine >>> \n>>> method used is AsymptoticLimits\n>>> random number generator seed is 123456\n\n -- AsymptoticLimits ( CLs ) --\nObserved Limit: r < 1.6281\nExpected  2.5%: r < 0.9640\nExpected 16.0%: r < 1.4329\nExpected 50.0%: r < 2.3281\nExpected 84.0%: r < 3.9800\nExpected 97.5%: r < 6.6194\n\nDone in 0.00 min (cpu), 0.00 min (real)\n\nreal\t0m11.151s\nuser\t0m5.027s\nsys\t0m2.497s',
 '/srv/temp\n <<< Combine >>> \n>>> method used is AsymptoticLimits\n>>> random number generator seed is 123456\n\n -- AsymptoticLimits ( CLs ) --\nObserved Limit: r < 1.6281\nExpected  2.5%: r < 0.9640\nExpected 16.0%: r < 1.4329\nExpected 50.0%: r < 2.3281\nExpected 84.0%: r < 3.9800\nExpected 97.5%: r < 6.6194\n\nDone in 0.00 min (cpu), 0.00 min (real)\n\nreal\t0m11.362s\nuser\t0m5.695s\nsys\t0m2.840s',
 '/srv/temp\n <<< Combine >>> \n>>> method used is AsymptoticLimits\n>>> random number generator seed is 123456\n\n -- AsymptoticLimits ( CLs ) --\nObserved Limit: r < 1.6281\nExpe

In [ ]:
def inc(x):
    return x + 1

def add(x, y):
    return x + y

a = client.submit(inc, 10)  # calls inc(10) in background thread or process
b = client.submit(inc, 20)

In [ ]:
a.result()

In [ ]:
b.result()

In [ ]:
futures = client.map(inc, range(1000))

In [ ]:
results = client.gather(futures)

In [ ]:
results

### set up root locally for testing

In [ ]:
%%bash 
source /cvmfs/cms.cern.ch/cmsset_default.sh 
cd /cvmfs/cms.cern.ch/slc7_amd64_gcc700/cms/cmssw/CMSSW_10_5_0/ 
eval `scramv1 runtime -sh` 
cd - 

cd /home/users/hmei/tmp/HiggsAnalysis/CombinedLimit
#cd HiggsAnalysis/CombinedLimit/
source ./env_standalone.sh 
cd -

which combine
#time python doScan_testDask.py

In [ ]:
import multiprocessing
import time
from datetime import datetime

def fill(l, num):
    #l[num] = num
    #print (num)
    time.sleep(5)
    dateTimeObj = datetime.now()
    print (dateTimeObj)
    
    #print (multiprocessing.current_process().name)
def print_name(name):
    print (name)
    dateTimeObj = datetime.now()
    print (dateTimeObj)

jobs = []
for i in range(10):
    p = multiprocessing.Process(name="fit_" + str(i),target=fill,args=(l_,i))
    jobs.append(p)
    p.start()


for job in jobs:
    job.join()
    
#while len(jobs) > 0:
#      jobs = [job for job in jobs if job.is_alive()]
#      time.sleep(1)

q = multiprocessing.Process(target=print_name, args=("bob",))
q.start()
#q.join()


In [ ]:
import uproot

In [ ]:
file = uproot.open("/hadoop/cms/store/user/smay/ttH/FinalFitTrees/ttHHadronic__v3.10_8Oct2019_RunII_MVA_Presel_impute_addDNNs_addTopTag_FinalFitTree.root")
file

In [ ]:
file.keys()
#file["t"]
#file["t"].keys()
#file["t"].values()
file.compression

In [ ]:
tree = file["t"]
tree.show()

In [ ]:
tree.name, tree.title, tree.numentries

In [ ]:
a1 = tree.array("tth_runII_mva")
print (a1)
tree["tth_runII_mva"].interpretation

In [ ]:
tree.arrays(["run","mass"])